In [159]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import imageio
import numpy as np
import warnings

In [4]:
tracking1=pd.read_csv('tracking_week_1.csv')
plays=pd.read_csv('plays.csv')

tracking1.loc[tracking1['playDirection'] == 'left', 'x'] = 120 - tracking1.loc[tracking1['playDirection'] == 'left', 'x']
tracking1.loc[tracking1['playDirection'] == 'left', 'y'] = (160/3) - tracking1.loc[tracking1['playDirection'] == 'left', 'y']
tracking1.loc[tracking1['playDirection'] == 'left', 'dir'] += 180
tracking1.loc[tracking1['dir'] > 360, 'dir'] -= 360
tracking1.loc[tracking1['playDirection'] == 'left', 'o'] += 180
tracking1.loc[tracking1['o'] > 360, 'o'] -= 360

tracking1_with_plays = tracking1.merge(plays, on=['gameId', 'playId'], how='left')
tracking1_with_plays = tracking1_with_plays[tracking1_with_plays['playNullifiedByPenalty'] == 'N']
tracking1_with_plays['is_on_offense'] = tracking1_with_plays['club'] == tracking1_with_plays['possessionTeam']
tracking1_with_plays['is_on_defense'] = tracking1_with_plays['club'] == tracking1_with_plays['defensiveTeam']
tracking1_with_plays['is_ballcarrier'] = tracking1_with_plays['ballCarrierId'] == tracking1_with_plays['nflId']

tracking1_with_plays

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,is_on_offense,is_on_defense,is_ballcarrier
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,31.630000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,31.530000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,31.440000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,31.360000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,31.280000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,2022-09-12 23:05:57.799999,NaN,football,left,63.779999,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407435,2022091200,3826,NaN,football,50,2022-09-12 23:05:57.900000,NaN,football,left,63.939999,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407436,2022091200,3826,NaN,football,51,2022-09-12 23:05:58.000000,NaN,football,left,64.110001,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407437,2022091200,3826,NaN,football,52,2022-09-12 23:05:58.099999,NaN,football,left,64.270000,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False


In [55]:
tracking1_with_plays['o']

0          51.74
1          50.98
2          50.98
3          52.38
4          53.36
           ...  
1407434      NaN
1407435      NaN
1407436      NaN
1407437      NaN
1407438      NaN
Name: o, Length: 1370386, dtype: float64

In [80]:
def preprocessing(df, past=False):
    df['radiansDirection'] = df['dir'].astype(float).apply(math.radians) #Converts angle in degrees to radians
    df['xComponent']=df['radiansDirection'].astype(float).apply(math.cos) #Converts angle into an x and y component
    df['yComponent']=df['radiansDirection'].astype(float).apply(math.sin)
    df['xspeed']=df['xComponent']*df['s'] #Determines magnitude of speed by multiplying x and y component by magnitude of speed
    df['yspeed']=df['yComponent']*df['s']
    if past:
        df['OLD_x']=df['x']
        df['OLD_y']=df['y']
        df['OLD_xspeed']=df['xspeed']
        df['OLD_yspeed']=df['yspeed']
        df['OLD_o']=df['o']
        df['OLD_s']=df['s']
        df['OLD_dir']=df['dir']
        df['PROJ_x']=df['OLD_x']+df['OLD_xspeed']*0.1
        df['PROJ_y']=df['OLD_y']+df['OLD_yspeed']*0.1
        return df[['gameId', 'playId', 'nflId', 'OLD_x', 'OLD_y', 'OLD_s', 'OLD_o', 'OLD_dir', 'OLD_xspeed', 'OLD_yspeed', 'PROJ_x', 'PROJ_y']]
    return df

In [118]:
def model(df, nflId):
    if nflId==38577:
        return 1
    return 0

In [150]:
def generate_counter_factuals(tracking, gameId, playId, time, prev_locations):
    tracking_single=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==time) & (tracking['gameId']==gameId) & (tracking['nflId'].isna()==False)]
    tracking_single=preprocessing(tracking_single)
    tracking_single=tracking_single.merge(prev_locations, how='inner', on=['gameId', 'playId', 'nflId'])
    defensive_ids=tracking_single.loc[tracking_single['is_on_defense']==True]['nflId']
    tracking_single['yards_saved']=0
    regular_prediction=model(tracking_single[['x', 'y', 's', 'dir', 'o', 'is_on_offense', 'is_on_defense', 'is_ballcarrier']], 0)
    for i in defensive_ids:
        passed_df=tracking_single.copy()
        list_values=list(passed_df.loc[passed_df['nflId']==i].iloc[0][['PROJ_x', 'PROJ_y', 'OLD_s', 'OLD_dir', 'OLD_o']])
        passed_df.loc[passed_df['nflId']==i, ['x', 'y', 's', 'dir', 'o']]=list_values
        tracking_single.loc[tracking_single['nflId']==i, 'yards_saved']=model(passed_df[['x', 'y', 's', 'dir', 'o', 'is_on_offense', 'is_on_defense', 'is_ballcarrier']], i)-regular_prediction
    return tracking_single.loc[tracking_single['is_on_defense']==True][['nflId', 'yards_saved']]

In [157]:
def generate_counter_factual_movements(tracking, gameId, playId):
    distinctTimes=tracking.loc[(tracking['playId'] == playId)& (tracking['gameId']==gameId) ]['time'].unique()
    directory = "Play_"+str(playId)
    try:
        os.mkdir(directory)
    except:
        pass
    os.chdir(directory)
    previous_defense_time=distinctTimes[0]
    savedTable={}
    for i in range(1, len(distinctTimes[1:])):
        prev_location=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==previous_defense_time) & (tracking['gameId']==gameId)]
        prev_location=preprocessing(prev_location, True) 
        dfForRunning=generate_counter_factuals(tracking, gameId, playId, distinctTimes[i], prev_location)
        if i==1:
            savedTable=dfForRunning
        else:
            savedTable=savedTable.merge(dfForRunning, how='outer', on='nflId', suffixes=('_table1', '_table2'))
            savedTable['yards_saved']=savedTable['yards_saved_table1']+savedTable['yards_saved_table2']
            savedTable=savedTable[['nflId', 'yards_saved']]
        previous_defense_time=distinctTimes[i]
    return savedTable

In [160]:
warnings.filterwarnings('ignore')

generate_counter_factual_movements(tracking1_with_plays, 2022090800, 56)

,nflId,yards_saved
0,38577.0,20
1,41239.0,0
2,42816.0,0
3,43294.0,0
4,43298.0,0
5,43335.0,0
6,47844.0,0
7,47917.0,0
8,48026.0,0
9,52607.0,0


In [ ]:
def processToVisualize(tracking, play, game_info, gameId, playId, time):
  tracking_single=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==time) & (tracking['gameId']==gameId)]
  testingNew=pd.merge(tracking_single, play, on=['gameId', 'playId'], how='inner')
  testingNew=pd.merge(testingNew, game_info, on=['gameId'], how='inner')
  testingNew['radiansDirection'] = testingNew['dir'].astype(float).apply(math.radians) #Converts angle in degrees to radians
  testingNew['xComponent']=testingNew['radiansDirection'].astype(float).apply(math.cos) #Converts angle into an x and y component
  testingNew['yComponent']=testingNew['radiansDirection'].astype(float).apply(math.sin)
  testingNew['xspeed']=testingNew['xComponent']*testingNew['s'] #Determines magnitude of speed by multiplying x and y component by magnitude of speed
  testingNew['yspeed']=testingNew['yComponent']*testingNew['s']
  return testingNew

In [ ]:
def generate_predictions_df